In [3]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/train'
validation_dir = 'C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/valid'

batch_size = 20
img_size = 224

train_datagen = ImageDataGenerator(rescale=1/255)

# train_datagen = ImageDataGenerator(rescale=1/255,
#                                    rotation_range=20,  # 지정된 각도 범위내에서 임의로 원본 이미지를 회전
#                                    width_shift_range=0.1,
#                                    height_shift_range=0.1,
#                                    zoom_range=0.1,    #  1-수치 혹은 1+수치만큼 확대 및 축소
#                                    horizontal_flip=True,
#                                    vertical_flip=True)



validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size,img_size),                      
    batch_size=batch_size,     
    class_mode='categorical'                    
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size,img_size),                       
    batch_size=batch_size,    
    class_mode='categorical'                    
)

Found 21017 images belonging to 6 classes.
Found 5256 images belonging to 6 classes.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.applications import VGG16

np.random.seed(42)

# 모델 구축

# vgg16 모델 불러오기
pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
pre_trained_vgg.trainable = False
pre_trained_vgg.summary()

# vgg16 모델에 덧붙이기
model = Sequential()
model.add(pre_trained_vgg)
model.add(Flatten())
model.add(Dense(4096, activation='relu', kernel_initializer='he_normal'))
#model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu', kernel_initializer='he_normal'))
#model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu', kernel_initializer='he_normal'))

model.add(Dense(6, activation='softmax'))

model.summary()

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy',
                metrics=['accuracy'])

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# 데이터 개수 / batch_size
steps_per_epoch = 21017 / batch_size
val_steps = 5256 / batch_size

history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=30,
                    validation_data=validation_generator,
                    validation_steps=val_steps,
                    callbacks=[early_stopping],
                    verbose=1
                    )

Epoch 1/30
1051/1050 [==============================] - 866s 824ms/step - loss: 0.0812 - accuracy: 0.9784 - val_loss: 0.0316 - val_accuracy: 0.9922
Epoch 2/30
1051/1050 [==============================] - 756s 719ms/step - loss: 0.0248 - accuracy: 0.9935 - val_loss: 0.0263 - val_accuracy: 0.9932
Epoch 3/30
1051/1050 [==============================] - 744s 708ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 0.0372 - val_accuracy: 0.9899
Epoch 4/30
1051/1050 [==============================] - 727s 691ms/step - loss: 0.0080 - accuracy: 0.9978 - val_loss: 0.0337 - val_accuracy: 0.9914
Epoch 5/30
1051/1050 [==============================] - 734s 699ms/step - loss: 0.0065 - accuracy: 0.9983 - val_loss: 0.0344 - val_accuracy: 0.9933


In [7]:
model.save('C:/Users/s_csmscox/jupyterSave/kFashion/vgg16.h5')

In [8]:
import pathlib
import os,glob
import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical

directory = 'C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/test'
categories = ['0', '1', '2', '3', '4', '5']
nb_classes = len(categories)

w = img_size
h = img_size

X = []
y = []

for idx, obj in enumerate(categories):

    image_dir = directory + "/" + obj
    files = glob.glob(image_dir+"/*.jpg")
    print(obj, " 파일 길이 : ", len(files))
    
    for i, f in enumerate(files):
        img = Image.open(f)
        
        # 사이즈 조절
        img = img.resize((w, h))
        
        img = np.asarray(img)
        img = img.reshape(w,h,3)

        X.append(img)
        y.append(idx)

        if i % 10000 == 0:
            print(obj, " : ", f)
            
X_test = np.array(X)
X_test = X_test / 255

y_test = np.array(y)
y_test = to_categorical(y_test)

0  파일 길이 :  3081
0  :  C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/test/0\10003.jpg
1  파일 길이 :  538
1  :  C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/test/1\10018.jpg
2  파일 길이 :  1469
2  :  C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/test/2\10080.jpg
3  파일 길이 :  611
3  :  C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/test/3\10202.jpg
4  파일 길이 :  509
4  :  C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/test/4\10105.jpg
5  파일 길이 :  362
5  :  C:/Users/s_csmscox/jupyterSave/kFashion/kaggle_fashion/test/5\13564.jpg


In [9]:
# 모델 평가
print("-- Evaluate --")
scores = model.evaluate(X_test, y_test)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
206/206 [==============================] - 14s 67ms/step - loss: 0.0391 - accuracy: 0.9922
accuracy: 99.22%


In [6]:
from sklearn.metrics import classification_report

#y_pred와 y_test를 일차원 배열로 만 드는 함수
def y_to_list(y_pred, y_test):
    yp = []
    yt = []
    for i in y_pred:
        idx = 0
        m = 0
        for j in range(0,21):
            if(i[j]>m):
                m = i[j]
                idx = j
        yp.append(idx)
    
    for i in y_test:
        idx = 0
        for j in range(0,21):
            if(i[j]==1):
                idx = j
                break
        yt.append(idx)
    
    return yp, yt

#모델 예측 및 classification_report를 통한 평가
y_pred = model.predict(X_test)

yp, yt = y_to_list(y_pred, y_test)

print(classification_report(yt, yp))

              precision    recall  f1-score   support

           0       0.52      0.36      0.42       292
           1       0.66      0.76      0.71       742
           2       0.00      0.00      0.00       790
           3       0.00      0.00      0.00       346
           4       0.00      0.00      0.00       221
           5       0.00      0.00      0.00         7
           6       0.03      0.03      0.03        94
           7       0.00      0.00      0.00       335
           8       0.02      0.00      0.00       713
           9       0.00      0.00      0.00       383
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00        28
          12       0.00      0.00      0.00       220
          13       0.00      0.00      0.00       226
          14       0.00      0.02      0.01       118
          15       0.00      0.00      0.00        58
          16       0.00      0.00      0.00        82
          17       0.02    

C:\Users\s_csmscox\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\s_csmscox\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
